In [1]:
import sklearn
import os 
import pandas as pd
import numpy as np

**IMPORTING FILES**

In [2]:
#reads files and creates list
def filereader(path):
    list_of_comments = []
    all_files = os.listdir(path)
    for folder in all_files:
        files = os.listdir(path+folder)
        for file in files:
            pathname = path+folder+'\\'+file
            file = open(pathname, 'r')
            list_of_comments.append(file.readline())
    return list_of_comments

In [3]:
neg_pol_truthful = filereader("C:\\Users\\aligthar\\Desktop\\negative_polarity\\truthful_from_Web\\")
neg_pol_deceptive = filereader("C:\\Users\\aligthar\\Desktop\\negative_polarity\\deceptive_from_MTurk\\")
pos_pol_truthful = filereader("C:\\Users\\aligthar\\Desktop\\positive_polarity\\truthful_from_TripAdvisor\\")
pos_pol_deceptive = filereader("C:\\Users\\aligthar\\Desktop\\positive_polarity\\deceptive_from_MTurk\\")

**PREPROCESSING**

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from random import shuffle

In [5]:
def preprocessing(text, pos_tags=False):
    # tokenize into words
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]   

    # remove stopwords
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]

    # remove words less than three characters
    tokens = [word for word in tokens if len(word) >= 3]

    # remove capitalization
    tokens = [word.lower() for word in tokens]

    # lemmatizing
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(word) for word in tokens]

    # includes POS_tags
    if pos_tags:
        tokens = nltk.pos_tag(tokens)
        tokens = ['_'.join(t) for t in tokens]
    
    preprocessed_text= ' '.join(tokens)

    return preprocessed_text 

def preprocesser(list_of_texts): # turning lists into preprocessed lists
    new = []
    for item in list_of_texts:
        new += [preprocessing(item)]
    return new

In [6]:
# labelling data: 0 for truthful, 1 for deceptive
corpus_neg_truthful = [(r,0) for r in preprocesser(neg_pol_truthful)]
corpus_neg_deceptive = [(r,1) for r in preprocesser(neg_pol_deceptive)]
corpus_pos_truthful = [(r,0) for r in preprocesser(pos_pol_truthful)]
corpus_pos_deceptive = [(r,1) for r in preprocesser(pos_pol_deceptive)]

In [7]:
# shuffle data instances
from random import shuffle
corpus = corpus_neg_truthful+corpus_neg_deceptive+corpus_pos_truthful+corpus_pos_deceptive
shuffle(corpus)
reviews = [r for r,l in corpus]
labels_y = [l for r,l in corpus]

**LABEL PROPAGATION**

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from statistics import mean, stdev
from datetime import datetime

In [12]:
def main_labelX_CV(ratio, vectorizertype, ngram, nsplits):
   
    """
    Main function with cross-validation included
    Parameters:
        ratio (number of labels to remove) = float (0.80 / 0.90 / 0.95)
        vectorizertype (word counts or TF-IDF values) = CountVectorizer / TfidfVectorizer
        ngram (word pairs) = int (1=unigram, 2=bigram)
        nsplits (cross validation splits) = int (number of splits)
    
    """
    # initialize results dictionary     
    LP_results = {'accuracy':[], '1':{'precision':[], 'recall':[], 'f1-score':[]}}
    LS_results = {'accuracy':[], '1':{'precision':[], 'recall':[], 'f1-score':[]}}
    
    
    # creating dataframe 
    vectorizer = vectorizertype(min_df=2, max_df=0.5, ngram_range=(1,ngram))
    X = vectorizer.fit_transform(reviews)
    DF = pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names())
    DF['Y'] = labels_y
    X = DF.drop('Y', axis=1)
    y = DF['Y']
    
    # feature selection
    def chi_square(X, y): 
        F, pval = chi2(X, y) 
        return F
    def feature_ranking(F):
        idx = np.argsort(F)
        return idx[::-1]
    df = pd.DataFrame(feature_ranking(chi_square(X,y)))
    df.columns = ['column_numbers']
    clms_toselect = df['column_numbers'].head(1000)
    a = [DF.columns[i] for i in clms_toselect]
    X = DF[a]
    X = X.to_numpy()
    
    # KFold crossvalidation
    kf = KFold(n_splits=nsplits, shuffle=False)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # removing labels    
        rng = np.random.RandomState(42) 
        random_unlabeled_points = rng.rand(len(y_train)) < ratio # % ratio KNOB
        labels = np.copy(y_train)
        labels[random_unlabeled_points] = -1
        y_train = labels

        # train models
        label_prop_model = LabelPropagation(kernel='knn', n_neighbors=200)        
        label_prop_model.fit(X_train, y_train)

        label_spread_model = LabelSpreading(kernel='knn', n_neighbors=200)    
        label_spread_model.fit(X_train, y_train)

        # evaluate models
        y_pred_LP = label_prop_model.predict(X_test)
        y_pred_LS = label_spread_model.predict(X_test)
        
        report = classification_report(y_test,y_pred_LP, output_dict=True)
        LP_results['accuracy'] += [report['accuracy']]
        LP_results['1']['precision'] += [report['1']['precision']]
        LP_results['1']['recall'] += [report['1']['recall']]
        LP_results['1']['f1-score'] += [report['1']['f1-score']]
    
        report = classification_report(y_test,y_pred_LS, output_dict=True)     
        LS_results['accuracy'] += [report['accuracy']]
        LS_results['1']['precision'] += [report['1']['precision']]
        LS_results['1']['recall'] += [report['1']['recall']]
        LS_results['1']['f1-score'] += [report['1']['f1-score']]

    with open('Label_Prop_Spread_results.txt', 'a') as file:
        file.write('___ratio:' + str(ratio) + '|ngram:' + str(ngram) + '|vectorizer:'+ str(vectorizertype) + '___\n')
        file.write('Label Propagation:\n')
        file.write('accuracy: ' + str(round(mean(LP_results['accuracy']),2)) + '\n')
        file.write('stddev: ' + str(stdev(LP_results['accuracy'])) + '\n')
        file.write('precision: ' + str(round(mean(LP_results['1']['precision']),2)) + '\n')
        file.write('recall: ' + str(round(mean(LP_results['1']['recall']),2)) + '\n')
        file.write('F1 :' + str(round(mean(LP_results['1']['f1-score']),2)) + '\n\n')

        file.write('Label Spreading:\n')        
        file.write('accuracy: ' + str(round(mean(LS_results['accuracy']),2)) + '\n')
        file.write('stddev: ' + str(stdev(LS_results['accuracy'])) + '\n')
        file.write('precision: ' + str(round(mean(LS_results['1']['precision']),2)) + '\n')
        file.write('recall: ' + str(round(mean(LS_results['1']['recall']),2)) + '\n')
        file.write('F1 :' + str(round(mean(LS_results['1']['f1-score']),2)) + '\n\n')
        obj = datetime.now()
        file.write(str(obj) + '\n\n\n')

In [13]:
main_labelX_CV(ratio=0.80, vectorizertype=CountVectorizer, ngram=1, nsplits=5)

C:\Users\aligthar\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aligthar\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aligthar\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
